In [25]:
!pip install qiskit
!pip install qiskit_ibm_runtime
!pip install qiskit qiskit-ibm-provider numpy
!pip install qiskit-aer

In [27]:
from qiskit import QuantumCircuit, transpile, transpile
from qiskit_aer import Aer


def encode_message(bits, bases):
    circuits = []
    for bit, basis in zip(bits, bases):
        qc = QuantumCircuit(1, 1)
        if basis == 0:  # Z basis
            if bit == 1:
                qc.x(0)
        else:  # X basis
            if bit == 0:
                qc.h(0)
            else:
                qc.x(0)
                qc.h(0)
        circuits.append(qc)
    return circuits

def measure_message(circuits, bases):
    measured_circuits = []
    for qc, basis in zip(circuits, bases):
        measured_qc = qc.copy()
        if basis == 1:  # X basis → apply H before measurement
            measured_qc.h(0)
        measured_qc.measure(0, 0)
        measured_circuits.append(measured_qc)
    return measured_circuits

def remove_garbage(a_bases, b_bases, bits):
    return [bit for i, bit in enumerate(bits) if a_bases[i] == b_bases[i]]

def bb84_protocol(n_bits=8, seed=0):
    np.random.seed(seed)

    # Alice's random bits & bases
    alice_bits = np.random.randint(2, size=n_bits)
    alice_bases = np.random.randint(2, size=n_bits)

    # Bob's random bases
    bob_bases = np.random.randint(2, size=n_bits)

    # Encode + measure
    message = encode_message(alice_bits, alice_bases)
    bob_circuits = measure_message(message, bob_bases)

    # Run on local simulator
    backend = Aer.get_backend("qasm_simulator")
    transpiled = transpile(bob_circuits, backend)
    job = backend.run(transpiled, shots=1)

    results = job.result()
    bob_results = []
    for i in range(n_bits):
        counts = results.get_counts(i)
        outcome = list(counts.keys())[0]  # only one shot
        bob_results.append(int(outcome))

    # Sifted keys
    alice_key = remove_garbage(alice_bases, bob_bases, alice_bits)
    bob_key   = remove_garbage(alice_bases, bob_bases, bob_results)

    return {
        "alice_bits": alice_bits,
        "alice_bases": alice_bases,
        "bob_bases": bob_bases,
        "bob_results": bob_results,
        "alice_key": alice_key,
        "bob_key": bob_key
    }

# Example run
if __name__ == "__main__":
    results = bb84_protocol(n_bits=8, seed=42)
    print("Alice bits:   ", results['alice_bits'])
    print("Alice bases:  ", results['alice_bases'])
    print("Bob bases:    ", results['bob_bases'])
    print("Bob results:  ", results['bob_results'])
    print("Alice key:    ", results['alice_key'])
    print("Bob key:      ", results['bob_key'])

ERROR:stevedore.extension:Could not load 'ibm_backend': cannot import name 'ProviderV1' from 'qiskit.providers' (/usr/local/lib/python3.12/dist-packages/qiskit/providers/__init__.py)
ERROR:stevedore.extension:Could not load 'ibm_dynamic_circuits': cannot import name 'ProviderV1' from 'qiskit.providers' (/usr/local/lib/python3.12/dist-packages/qiskit/providers/__init__.py)
ERROR:stevedore.extension:Could not load 'ibm_backend': cannot import name 'ProviderV1' from 'qiskit.providers' (/usr/local/lib/python3.12/dist-packages/qiskit/providers/__init__.py)
ERROR:stevedore.extension:Could not load 'ibm_dynamic_circuits': cannot import name 'ProviderV1' from 'qiskit.providers' (/usr/local/lib/python3.12/dist-packages/qiskit/providers/__init__.py)


Alice bits:    [0 1 0 0 0 1 0 0]
Alice bases:   [0 1 0 0 0 0 1 0]
Bob bases:     [1 1 1 0 1 0 1 1]
Bob results:   [1, 1, 1, 0, 1, 1, 0, 1]
Alice key:     [np.int64(1), np.int64(0), np.int64(1), np.int64(0)]
Bob key:       [1, 0, 1, 0]
